In [1]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split


Preparing the data

In [49]:
# importing the data
item_train = pd.read_csv('item_train.csv')
user_train = pd.read_csv('user_train.csv')
y_train = pd.read_csv('y_train.csv')

num_user_features = user_train.shape[1] - 3  # remove userid, rating count and ave rating during training
num_item_features = item_train.shape[1] - 1  # remove movie id at train time
uvs = 3  # user genre vector start
ivs = 3  # item genre vector start
u_s = 3  # start of columns to use in training, user
i_s = 1  # start of columns to use in training, items
print(f"Number of training vectors: {len(item_train)}")



Number of training vectors: 100836


In [50]:
print("num_user_features", num_user_features)

num_user_features 19


In [51]:
# scale training data
item_train_unscaled = item_train
user_train_unscaled = user_train
y_train_unscaled    = y_train

scalarItem = StandardScaler()
scalarItem.fit(item_train)
item_train = scalarItem.transform(item_train)

scalarUser = StandardScaler()
scalarUser.fit(user_train)
user_train = scalarUser.transform(user_train)

# Convert y_train to a NumPy array and reshape it
y_train_array = y_train.values.reshape(-1, 1)


scalarTarget = MinMaxScaler((-1,1))
scalarTarget.fit(y_train_array)
y_train= scalarTarget.transform(y_train_array)


print(np.allclose(item_train_unscaled, scalarItem.inverse_transform(item_train)))
print(np.allclose(user_train_unscaled, scalarUser.inverse_transform(user_train)))

False
True


In [52]:
item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)
print(f"movie/item training data shape: {item_train.shape}")
print(f"movie/item test data shape: {item_test.shape}")

movie/item training data shape: (80668, 22)
movie/item test data shape: (20168, 22)


In [53]:
print("Shape of user_train slice:", user_train[:, u_s:].shape)
print("Shape of item_train slice:", item_train[:, i_s:].shape)

Shape of user_train slice: (80668, 19)
Shape of item_train slice: (80668, 21)


In [54]:
from tensorflow.keras import layers

num_outputs = 32

tf.random.set_seed(1)

class L2NormalizationLayer(layers.Layer):
    def call(self, inputs):
        return tf.linalg.l2_normalize(inputs, axis=1)

user_NN = tf.keras.Sequential([
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(num_outputs),
])

item_NN = tf.keras.Sequential([
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(num_outputs),
])

# create the user input and point to the base network

input_user = tf.keras.Input(shape=(num_user_features,))
vu = user_NN(input_user)
vu = L2NormalizationLayer()(vu)

# create the item input and point to the base network

input_item = tf.keras.Input(shape=(num_item_features,))
vm = item_NN(input_item)
vm = L2NormalizationLayer()(vm)

# compute the dot product of the two vectors vu and vm

output = tf.keras.layers.Dot(axes=1)([vu, vm])

# specify the inputs and output of the model

model = tf.keras.Model(inputs=[input_user, input_item], outputs=output)

model.summary()


Model: "functional_18"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_33      │ (None, 19)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_35      │ (None, 21)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_38       │ (None, 32)        │     42,144 │ input_layer_33[0… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_39       │ (None, 32)        │     42,656 │ input_layer_35[0… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ l2_normalization_l… │ (None, 32)        │          0 │ sequential_38[0]… │
│ (L2NormalizationLa… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ l2_normalization_l… │ (None, 32)        │          0 │ sequential_39[0]… │
│ (L2NormalizationLa… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot_2 (Dot)         │ (None, 1)         │          0 │ l2_normalization… │
│                     │                   │            │ l2_normalization… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 84,800 (331.25 KB)

 Trainable params: 84,800 (331.25 KB)

 Non-trainable params: 0 (0.00 B)

In [65]:
tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=1.0)
model.compile(optimizer=opt,
              loss=cost_fn)

In [66]:
tf.random.set_seed(1)
model.fit([user_train[:, u_s:], item_train[:, i_s:]], y_train, epochs=100)

Epoch 1/100
2521/2521 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 0.1737
Epoch 2/100
2521/2521 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.1739
Epoch 3/100
2521/2521 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.1739
Epoch 4/100
2521/2521 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.1739
Epoch 5/100
2521/2521 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.1738
Epoch 6/100
2521/2521 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.1738
Epoch 7/100
2521/2521 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.1738
Epoch 8/100
2521/2521 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.1738
Epoch 9/100
2521/2521 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.1738
Epoch 10/100
2521/2521 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.1738
Epoch 11/100
2521/2521 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.1738
Epoch 12/100
2521/2521 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.1738
Epoch 13/100
2521/2521 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.1737
Epoch 14/100
2521/2521 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.1737
Epoch 15/100
25